# Preparo da massa de áudios em formato WAV

### Referências

https://medium.com/@keur.plkar/audio-data-augmentation-in-python-a91600613e47

https://docs.python.org/3/library/argparse.html

https://www.w3schools.com/python/python_try_except.asp

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import librosa
import librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
# diretório de entrada
DIR_ENTRADA = 'mp3'

# diretório de saída
DIR_SAIDA = 'wavbal5'
DIR_SAIDA2 = 'wavbal5a'

# duração de cada trecho de áudio
DURACAO_TRECHO = 5 # segundos
SAMPLE_RATE = 16000 # 16 kHz

# calcular a quantidade de pontos na onda
PONTOS_ONDA = SAMPLE_RATE * DURACAO_TRECHO # 16 kHz * 5 s = 80000

## Verificar espécies disponíveis

In [3]:
ARQUIVO_ESPECIES = 'especies-oiapoque.txt'

especies = []
with open(ARQUIVO_ESPECIES, 'r') as f:
    especies = [especie.rstrip().lower().replace(' ', '_') for especie in f.readlines()]
especies

['ramphocelus_carbo',
 'pitangus_sulphuratus',
 'querula_purpurata',
 'cacicus_cela',
 'pteroglossus_aracari',
 'tangara_episcopus',
 'dacnis_cayana',
 'ramphastos_tucanus',
 'tangara_palmarum',
 'pionus_fuscus',
 'trogon_melanurus',
 'glyphorynchus_spirurus',
 'cyclarhis_gujanensis',
 'lipaugus_vociferans',
 'pionites_melanocephalus',
 'aramides_cajaneus',
 'ramphastos_vitellinus',
 'cathartes_aura',
 'turdus_leucomelas',
 'amazona_farinosa',
 'thamnophilus_punctatus',
 'thamnomanes_ardesiacus',
 'euphonia_violacea',
 'pithys_albifrons',
 'tachycineta_albiventer',
 'mionectes_macconnelli',
 'hypocnemis_cantator',
 'leptotila_rufaxilla',
 'dacnis_lineata',
 'tyrannus_melancholicus',
 'trogon_viridis',
 'manacus_manacus',
 'geotrygon_montana',
 'piaya_cayana',
 'myiopagis_gaimardii',
 'chloroceryle_americana',
 'trogon_violaceus',
 'bucco_capensis',
 'myrmotherula_surinamensis',
 'myiarchus_ferox',
 'phaethornis_sp',
 'thamnophilus_murinus',
 'dryocopus_lineatus',
 'formicarius_colma',


In [44]:
arquivos = []
especies_disponiveis = []

for especie in especies:
    dir_especie = os.path.join(DIR_ENTRADA, especie)
    if os.path.isdir(dir_especie):
        arquivos.append(len(os.listdir(dir_especie)) - 1)
        especies_disponiveis.append(especie)

especies = especies_disponiveis

In [45]:
pd.set_option('display.max_rows', 100)

df = pd.DataFrame({
    'especie': especies,
    'arquivos_mp3': arquivos
})

df.sort_values('especie')

,especie,arquivos_mp3
19,amazona_farinosa,46
15,aramides_cajaneus,37
54,attila_spadiceus,250
62,automolus_infuscatus,120
37,bucco_capensis,52
3,cacicus_cela,293
69,campephilus_rubricollis,120
17,cathartes_aura,7
50,celeus_undatus,25
53,ceratopipra_erythrocephala,147


## Recriar diretórios de saída - Passo 1

## Processar arquivos de áudio - Passo 1

## Calcular quantidade de arquivos gerados

In [46]:
arquivos = []
for especie in especies:
    dir_especie = os.path.join(DIR_SAIDA, especie)
    arquivos.append(len(os.listdir(dir_especie)))
#arquivos

In [47]:
df['arquivos_wav'] = arquivos
#maior_qtde = max(df['arquivos_wav'])
#df['diferenca'] = maior_qtde - df['arquivos_wav']
#df['multiplicacao'] = df[['diferenca', 'arquivos_mp3']].apply(
#    lambda x: np.round(x[0] / x[1], 0), axis=1)
df['rendimento'] = df['arquivos_wav'] / df['arquivos_mp3']
#df['mp3_necessarios'] = df['diferenca'] / df['rendimento']
#df['variacoes_mp3'] = df['mp3_necessarios'] / df['arquivos_mp3']
df.sort_values('especie')

,especie,arquivos_mp3,arquivos_wav,rendimento
19,amazona_farinosa,46,870,18.913043
15,aramides_cajaneus,37,558,15.081081
54,attila_spadiceus,250,2484,9.936000
62,automolus_infuscatus,120,910,7.583333
37,bucco_capensis,52,485,9.326923
3,cacicus_cela,293,3924,13.392491
69,campephilus_rubricollis,120,772,6.433333
17,cathartes_aura,7,85,12.142857
50,celeus_undatus,25,123,4.920000
53,ceratopipra_erythrocephala,147,1266,8.612245


In [48]:
df['arquivos_wav'].mean(), df['arquivos_wav'].std(), df['arquivos_wav'].median()

(927.6962025316456, 942.187271853003, 591.0)

In [49]:
limiar = round(df['arquivos_wav'].median())
print("limiar:", limiar)

df['wav_faltantes'] = df['arquivos_wav'].apply(lambda x: limiar - x if limiar > x else 0)
df

limiar: 591


,especie,arquivos_mp3,arquivos_wav,rendimento,wav_faltantes
0,ramphocelus_carbo,184,1220,6.630435,0
1,pitangus_sulphuratus,555,3637,6.553153,0
2,querula_purpurata,155,1071,6.909677,0
3,cacicus_cela,293,3924,13.392491,0
4,pteroglossus_aracari,60,382,6.366667,209
5,tangara_episcopus,0,0,NaN,591
6,dacnis_cayana,75,437,5.826667,154
7,ramphastos_tucanus,228,2498,10.956140,0
8,tangara_palmarum,0,0,NaN,591
9,pionus_fuscus,45,236,5.244444,355


## Recriar diretórios de saída - Passo 2

In [43]:
# criar diretórios de saída

if not os.path.isdir(DIR_SAIDA2):
    os.mkdir(DIR_SAIDA2)
print(DIR_SAIDA2)

for especie in especies:
    dir_especie2 = os.path.join(DIR_SAIDA2, especie)
    if not os.path.isdir(dir_especie2):
        os.mkdir(dir_especie2)
    print(dir_especie2)

wavbal5a
wavbal5a/ramphocelus_carbo
wavbal5a/pitangus_sulphuratus
wavbal5a/querula_purpurata
wavbal5a/cacicus_cela
wavbal5a/pteroglossus_aracari
wavbal5a/tangara_episcopus
wavbal5a/dacnis_cayana
wavbal5a/ramphastos_tucanus
wavbal5a/tangara_palmarum
wavbal5a/pionus_fuscus
wavbal5a/trogon_melanurus
wavbal5a/glyphorynchus_spirurus
wavbal5a/cyclarhis_gujanensis
wavbal5a/lipaugus_vociferans
wavbal5a/pionites_melanocephalus
wavbal5a/aramides_cajaneus
wavbal5a/ramphastos_vitellinus
wavbal5a/cathartes_aura
wavbal5a/turdus_leucomelas
wavbal5a/amazona_farinosa
wavbal5a/thamnophilus_punctatus
wavbal5a/thamnomanes_ardesiacus
wavbal5a/euphonia_violacea
wavbal5a/pithys_albifrons
wavbal5a/tachycineta_albiventer
wavbal5a/mionectes_macconnelli
wavbal5a/hypocnemis_cantator
wavbal5a/leptotila_rufaxilla
wavbal5a/dacnis_lineata
wavbal5a/tyrannus_melancholicus
wavbal5a/trogon_viridis
wavbal5a/manacus_manacus
wavbal5a/geotrygon_montana
wavbal5a/piaya_cayana
wavbal5a/myiopagis_gaimardii
wavbal5a/chloroceryle_

## Gerar arquivos adicionais com deslocamento - Passo 2

In [54]:
def produzir_arquivos_adicionais(especie, qtde_necessaria):

    if not especie:
        return 0
    
    dir_especie = os.path.join(DIR_SAIDA, especie)
    qtde_existente = len(os.listdir(dir_especie))
    qtde_faltante = (qtde_necessaria - qtde_existente if qtde_necessaria > qtde_existente else 0)
    
    print("[%s] iniciado (necessaria: %d, existentes: %d, faltantes: %d)" % (
        especie, qtde_necessaria, qtde_existente, qtde_faltante))
    qtde_arquivos = 0
    
    variacao = 0

    while (qtde_arquivos < qtde_faltante):
        variacao += 1

        # ler arquivos de entrada em MP3
        for arquivo in os.listdir(os.path.join(DIR_ENTRADA, especie)):

            # sair do laço se atingir a quantidade necessária
            if qtde_arquivos >= qtde_faltante:
                break
            
            # montar nome do arquivo de entrada
            entrada = os.path.join(DIR_ENTRADA, especie, arquivo)
            if not entrada.endswith('.mp3'):
                continue
            #print("entrada:", entrada)

            prefixo_saida = os.path.join(DIR_SAIDA2, especie, arquivo.replace('.mp3', ''))
            #print("prefixo_saida:", prefixo_saida)

            # carregar áudio original em formato MP3
            try:
                y, sr = librosa.load(entrada, sr=SAMPLE_RATE, mono=True)
            except:
                print("Erro ao ler arquivo:", entrada)
                continue
        
            #numero_pontos = y.shape[0]
            #print("-> numero_pontos:", numero_pontos)
            duracao = y.shape[0] / sr
            #print("-> duracao:", duracao, "seg")

            # calcular o número de cortes a serem efetuados no áudio
            numero_cortes = round(duracao / DURACAO_TRECHO)
            #print("-> numero_cortes:", numero_cortes)

            # deslocar o sinal (roll)
            tamanho_roll = variacao * int(PONTOS_ONDA / (variacao + 1))
            #print("--> tamanho_roll:", tamanho_roll)
            yr = np.roll(y, tamanho_roll)
            
            # adicionar ruído ao sinal
            yrs = yr + 0.009 * np.random.normal(0, 1, len(yr))
            
            for corte in np.arange(1, numero_cortes + 1):

                # sair do laço se atingir a quantidade necessária
                if qtde_arquivos >= qtde_faltante:
                    break
                    
                saida = "%s-%02d-%03d.wav" % (prefixo_saida, variacao, corte)
                    
                # calcular trechos de início e fim no áudio
                inicio = PONTOS_ONDA * (corte - 1)
                termino = PONTOS_ONDA * corte
                yrc = yrs[inicio:termino]
                
                duracao = yrc.shape[0] / sr
                if (duracao / DURACAO_TRECHO < 0.3):
                    break

                # gravar arquivo de áudio em formato WAV
                sf.write(saida, yrc, sr, format='wav', subtype='PCM_16')
                qtde_arquivos += 1
                if qtde_arquivos % 29 == 0:
                    print("--", saida)                

    print("[%s] finalizado -> qtde_arquivos: %d\n" % (especie, qtde_arquivos))
    return qtde_arquivos

In [58]:
%%time

ARQUIVOS_NECESSARIOS = 591

produzir_arquivos_adicionais('ramphocelus_carbo', ARQUIVOS_NECESSARIOS)
produzir_arquivos_adicionais('aramides_cajaneus', ARQUIVOS_NECESSARIOS)

[ramphocelus_carbo] iniciado (necessaria: 591, existentes: 1220, faltantes: 0)
[ramphocelus_carbo] finalizado -> qtde_arquivos: 0

[aramides_cajaneus] iniciado (necessaria: 591, existentes: 558, faltantes: 33)
-- wavbal5a/aramides_cajaneus/Aramides613991-01-004.wav
[aramides_cajaneus] finalizado -> qtde_arquivos: 33

CPU times: user 5.63 s, sys: 180 ms, total: 5.81 s
Wall time: 8.36 s


## Calcular quantidade de arquivos gerados ao final

In [ ]:
arquivos = []
for especie in especies:
    dir_especie = os.path.join(DIR_SAIDA2, especie)
    arquivos.append(len(os.listdir(dir_especie)))
#arquivos

In [ ]:
df['arquivos_wav2'] = arquivos
maior_qtde2 = max(df['arquivos_wav2'])
df['diferenca2'] = maior_qtde2 - df['arquivos_wav2']
df.sort_values('especie')